In [1]:
import numpy as np
import pandas as pd

In [2]:
# 1. data
sample = pd.read_csv("sample.csv")

In [3]:
sample = sample.iloc[:,1:]

In [4]:
sample

,outlook,temp,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


In [5]:
len(sample.loc[sample.play == 'yes',:])/len(sample)

0.6428571428571429

In [6]:
len(sample.loc[sample.play == 'no',:])/len(sample)

0.35714285714285715

In [7]:
pk_yes = np.sum(sample.play == 'yes')/len(sample)
pk_no = np.sum(sample.play == 'no')/len(sample)

In [8]:
-(pk_yes*np.log2(pk_yes) + pk_no*np.log2(pk_no))

0.9402859586706311

In [9]:
overcast_ = sample.loc[sample.outlook == 'overcast',:]

In [10]:
def entropy(x):
    px = np.unique(x,return_counts = True)[1]/len(x)
    entropy_ = -np.sum(px * np.log2(px))
    return entropy_

In [11]:
# y = play
total = entropy(sample.play)
# x = sample 독립변수

In [12]:
# data = sample
# x = temp
def info_gain(data,x,y):
    total = entropy(data[y])
    vals, counts = np.unique(data[x],return_counts=True)
    gain = [ count/len(data)*entropy(data.loc[data[x] == val, y ]) for val, count in zip(vals,counts)]
    return total - np.sum(gain)

In [14]:
info_gain(sample,'outlook','play')

0.24674981977443933

In [15]:
info_gain(sample,'temp','play')

0.02922256565895487

In [16]:
info_gain(sample,'humidity','play')

0.15183550136234159

In [17]:
info_gain(sample,'windy','play')

0.04812703040826949

12/28

In [19]:
x = np.array([1,1,1,1,3,2])

In [21]:
px = np.unique(x,return_counts=True)[1]/len(x)

In [24]:
np.sum(-px * np.log2(px))

1.2516291673878228

In [25]:
np.unique(sample['temp'],return_counts=True)

(array(['cool', 'hot', 'mild'], dtype=object), array([4, 4, 6], dtype=int64))

In [26]:
entropy(sample.play)

0.9402859586706311

In [46]:
# data : sample
# 종속변수(y) : play
# 독립(선택)변수(x1) : temp

sample.temp
# temp에 몇 종류가 있는지 unique 매개변수를 사용해서 파악 후 종류마다의 엔트로피를 구한다. np.unique(sample.temp) -> hot, cool, mild (3종류)
np.unique(sample.temp)[0] # cool일 경우만 먼저 구한다
x1 = sample.loc[sample['temp'] == np.unique(sample.temp)[0],'play'] # temp가 cool일 경우의 play만 구해서 x1로 지정
count_x1 = len(x1) # x1의 개수를 구한다.
entropy(x1) # x1의 엔트로피를 구한다.
np.unique(sample.temp)[1] # hot일 경우를 구한다.
x2 = sample.loc[sample['temp'] == np.unique(sample.temp)[1],'play'] # temp가 hot일 경우의 play만 구해서 x2로 지정
count_x2 = len(x2) # x2의 개수를 구한다.
entropy(x2) # x2의 엔트로피를 구한다.
np.unique(sample.temp)[2] # mild일 경우를 구한다.
x3 = sample.loc[sample['temp'] == np.unique(sample.temp)[2],'play'] # temp가 mild일 경우의 play만 구해서 x2로 지정
count_x3 = len(x3) # x3의 개수를 구한다.
entropy(x3) # x3의 엔트로피를 구한다.
entropy(x1), entropy(x2), entropy(x3), count_x1, count_x2, count_x3
#확률 값을 출력해본다.
px1 = count_x1/len(sample)
px2 = count_x2/len(sample)
px3 = count_x3/len(sample)
# 각 엔트로피에 대한 확률 값을 리스트로 만들어서 더한다.
gain = [px1 * entropy(x1), px2 * entropy(x2), px3 * entropy(x3)]
np.sum(gain)

0.9110633930116763

In [47]:
#이제 위에서 했던 것을 for문으로 만든다.
gain = []
for val in np.unique(sample.temp):
    x = sample.loc[sample['temp'] == val,'play']
    count_x = len(x)
    px = count_x/len(sample)
    gain.append(px*entropy(x))
np.sum(gain)

0.9110633930116763

In [64]:
# temp를 포함한 다른 선택변수도 적용할 수 있고, sample이 아닌 다른 데이터도 구할 수 있도록 함수로 구현한다.
def get_gain(data,x,y):
    total_info = entropy(data[y])
    gain = []
    for val in np.unique(data[x]):
        info = data.loc[data[x] == val,y]
        count_info = len(info)
        p_info = count_info/len(data)
        gain.append(p_info*entropy(info))
    return total_info - np.sum(gain)
# MAX에서 뺀 값이 변화가 가장 큰 것을 구하는 것 total_info - np.sum(gain)

In [65]:
# 각 선택변수를 적용시켜 가장 큰 값을 구하자.
get_gain(sample,'outlook','play'), get_gain(sample,'temp','play'), get_gain(sample,'humidity','play'), get_gain(sample,'windy','play')

(0.24674981977443933,
 0.02922256565895487,
 0.15183550136234159,
 0.04812703040826949)

In [75]:
# 각 선택변수를 알아서 적용시키게끔 for문으로 구현
features = sample.columns[:-1]
aa = []
for i in features:
    aa.append(get_gain(sample,i,'play'))
features[np.argmax(aa)]
# 가장 큰 값의 인덱스 값을 출력해서 변수명을 찾는다.

'outlook'

In [82]:
# 위에 했던 것도 함수화 합시다..
def max_gain(data,x,y):
    return x[np.argmax([get_gain(data,i,y) for i in x])]

In [83]:
max_gain(sample,sample.columns[:-1],sample.columns[-1])

'outlook'

In [84]:
# 그 동안 했던 거를 정리해봅시다.
# entropy
def entropy(x):
    px = np.unique(x,return_counts = True)[1]/len(x)
    entropy_ = -np.sum(px * np.log2(px))
    return entropy_
# get_gain
def get_gain(data,x,y):
    total_info = entropy(data[y])
    gain = []
    for val in np.unique(data[x]):
        info = data.loc[data[x] == val,y]
        count_info = len(info)
        p_info = count_info/len(data)
        gain.append(p_info*entropy(info))
    return total_info - np.sum(gain)
# max_gain
def max_gain(data,x,y):
    return x[np.argmax([get_gain(data,i,y) for i in x])]
# 이제 이 3개를 이용해서 tree를 만드려고합니다.

In [322]:
data = sample
x = sample.columns[:-1]
y = sample.columns[-1]
best_feature = max_gain(data,x,y)

In [165]:
# 여기까지는 sample에서 제일 좋은 값인 outlook을 구했고 outlook에서 트리를 만들어야겠죠?
tree = {best_feature:{}}

In [166]:
# 트리는 딕셔너리 형태가 좋아요
def make_tree(data,x,y):
    best_feature = max_gain(data,x,y)
    tree = {best_feature:{}}
    return tree

In [167]:
make_tree(data,x,y)

{'outlook': {}}

In [168]:
# best_feature가 아닌 나머지 제외된 값을 가져옵시다. 그리고 tree에 집어넣을게요
columns = x[x != best_feature]

In [235]:
def make_tree(data,x,y):
    best_feature = max_gain(data,x,y)
    tree = {best_feature:{}}
    return tree

In [170]:
make_tree(data,x,y)

{'outlook': {}}

In [171]:
# outlook에서의 변수들을 봅시다.
for val in np.unique(data[best_feature]):
    print(val)
vals = [val for val in np.unique(data[best_feature])]

overcast
rainy
sunny


In [172]:
data.loc[data[best_feature] == vals[0],:]

,outlook,temp,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes


In [389]:
tree = {best_feature:{}}

In [390]:
def make_tree(data,x,y):
    best_feature = max_gain(data,x,y)
    columns = x[x != best_feature]
    tree = {best_feature:{}}
    # leaf_node는 column이 없으므로 true면 node
    if columns.size > 0:
        for val in np.unique(data[best_feature]): # overcast, sunny, rainy
            sub_data = data.loc[data[best_feature] == val,:]
            sub_tree = make_tree(sub_data,columns,y)
            tree[best_feature][val] = sub_tree
    else: # leaf_node 일때
        if np.unique(data[y]).size == 1: # 하나일때(yes,no 중 하나만 있을때)
            tree[best_feature] = np.unique(data[y])[0]
        else: # 하나가 아니면 둘 중 많은거 구하기
            vals, counts = np.unique(data[y], return_counts=True)
            tree[best_feature]=vals[np.argmax(counts)]
    return tree

In [391]:
make_tree(data,x,y)

{'outlook': {'overcast': {'temp': {'cool': {'humidity': {'normal': {'windy': 'yes'}}},
    'hot': {'humidity': {'high': {'windy': 'yes'},
      'normal': {'windy': 'yes'}}},
    'mild': {'humidity': {'high': {'windy': 'yes'}}}}},
  'rainy': {'windy': {False: {'temp': {'cool': {'humidity': 'yes'},
      'mild': {'humidity': 'yes'}}},
    True: {'temp': {'cool': {'humidity': 'no'}, 'mild': {'humidity': 'no'}}}}},
  'sunny': {'humidity': {'high': {'temp': {'hot': {'windy': 'no'},
      'mild': {'windy': 'no'}}},
    'normal': {'temp': {'cool': {'windy': 'yes'},
      'mild': {'windy': 'yes'}}}}}}}

In [245]:
columns = x[x != best_feature]
sub_data = data.loc[data[best_feature] == 'sunny',:]
sub_tree = make_tree(sub_data,columns,y)
tree[best_feature]['sunny'] = sub_tree
tree

  outlook temp humidity  windy play
9   sunny  hot     high  False   no
   outlook temp humidity  windy play
10   sunny  hot     high   True   no
   outlook  temp humidity  windy play
11   sunny  mild     high  False   no
   outlook  temp humidity  windy play
12   sunny  cool   normal  False  yes
   outlook  temp humidity  windy play
13   sunny  mild   normal   True  yes


{'outlook': {'overcast': {'temp': {}}, 'rainy': {'windy': {}}, 'sunny': None}}

In [393]:
def get_tree(data,x,y,parent_class=None):
    try:
        if np.unique(data[y]).size == 1:
            return np.unique(data[y])[0]
        elif len(data) == 0:
            return 'case1'
        elif len(x) == 0:
            return parent_class
        best_feature = max_gain(data,x,y)
        columns = x[x != best_feature]
        tree = {best_feature:{}}
        for val in np.unique(data[best_feature]): # overcast, sunny, rainy
            vals, counts = np.unique(data.loc[data[best_feature] == val,y],return_counts=True)
            parent_class = vals[np.argmax(counts)]
            sub_data = data.loc[data[best_feature] == val,:]
            sub_tree = get_tree(sub_data,columns,y,parent_class)
            tree[best_feature][val] = sub_tree
        return tree
    except Exception as e:
        print(e)

In [420]:
get_tree(data,x,y)

{'outlook': {'overcast': 'yes',
  'rainy': {'windy': {'False': 'yes', 'True': 'no'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}

In [363]:
from pprint import pprint

In [367]:
pprint(make_tree(data,x,y))

{'outlook': {'overcast': {'temp': {'cool': {'humidity': {'normal': {'windy': 'yes'}}},
                                   'hot': {'humidity': {'high': {'windy': 'yes'},
                                                        'normal': {'windy': 'yes'}}},
                                   'mild': {'humidity': {'high': {'windy': 'yes'}}}}},
             'rainy': {'windy': {False: {'temp': {'cool': {'humidity': 'yes'},
                                                  'mild': {'humidity': 'yes'}}},
                                 True: {'temp': {'cool': {'humidity': 'no'},
                                                 'mild': {'humidity': 'no'}}}}},
             'sunny': {'humidity': {'high': {'temp': {'hot': {'windy': 'no'},
                                                      'mild': {'windy': 'no'}}},
                                    'normal': {'temp': {'cool': {'windy': 'yes'},
                                                        'mild': {'windy': 'yes'}}}}}}}


In [373]:
tmp = {
    'outlook' : ['overcast'],
    'temp' : ['hot'],
    'humidity' : ['normal'],
    'windy' : ['True'],
}
tmp = pd.DataFrame(tmp)

In [382]:
tmp1 = np.array(['rainy','mild','high','False'])

In [414]:
data.windy = data.windy.astype(str)

In [421]:
tree = make_tree(data,x,y)

In [422]:
tree['outlook'][tmp['outlook'][0]]['temp'][tmp['temp'][0]]['humidity'][tmp['humidity'][0]]

{'windy': 'yes'}

In [423]:
##
data.play = np.random.choice(['잘함','못함'],data.play.size)

In [425]:
tree = get_tree(data,x,y)

In [426]:
tree

{'temp': {'cool': {'outlook': {'overcast': '잘함',
    'rainy': '못함',
    'sunny': '잘함'}},
  'hot': '잘함',
  'mild': {'outlook': {'overcast': '못함',
    'rainy': {'humidity': {'high': {'windy': {'False': '잘함', 'True': '못함'}},
      'normal': '못함'}},
    'sunny': {'humidity': {'high': '못함', 'normal': '잘함'}}}}}}

In [429]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
model = DecisionTreeClassifier()

In [432]:
le_outlook = LabelEncoder().fit(data.outlook)
le_temp = LabelEncoder().fit(data.temp)
le_humidity = LabelEncoder().fit(data.humidity)
le_windy = LabelEncoder().fit(data.windy)

In [433]:
X = np.c_[le_outlook.transform(data.outlook),
         le_temp.transform(data.temp),
         le_humidity.transform(data.humidity),
         le_windy.transform(data.windy)]
y = data.play

In [434]:
model.fit(X,y)

DecisionTreeClassifier()

In [437]:
test_data = np.array(['overcast','cool','high','False'])

In [441]:
le_outlook.transform(['overcast']), le_temp.transform(['cool']), le_humidity.transform(['high']), le_windy.transform(['False'])

(array([0]), array([0]), array([0]), array([0]))

In [442]:
X.shape

(14, 4)

In [443]:
model.predict(np.array([0,0,0,0]).reshape(1,4))

array(['잘함'], dtype=object)

In [444]:
from sklearn import tree

In [445]:
from sklearn.tree import plot_tree

In [446]:
tree_model = DecisionTreeClassifier(max_depth=3)

In [452]:
from sklearn.naive_bayes import GaussianNB

In [453]:
model = GaussianNB()

In [454]:
model.fit(X,y)

GaussianNB()

In [455]:
model.predict(X)

array(['잘함', '잘함', '못함', '잘함', '못함', '잘함', '못함', '못함', '못함', '잘함', '잘함',
       '잘함', '잘함', '잘함'], dtype='<U2')

In [456]:
from sklearn import naive_bayes

In [458]:
dir(model)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_X',
 '_check_feature_names',
 '_check_n_features',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_joint_log_likelihood',
 '_more_tags',
 '_parameter_constraints',
 '_partial_fit',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_update_mean_variance',
 '_validate_data',
 '_validate_params',
 'class_count_',
 'class_prior_',
 'classes_',
 'epsilon_',
 'fit',
 'get_metadata_routing',
 'get_params'